In [12]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [4]:
mnist = input_data.read_data_sets("./mnist/", one_hot=True)

Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./mnist/t10k-images-idx3-ubyte.gz
Extracting ./mnist/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [8]:
print "data size ", mnist.train.num_examples, mnist.validation.num_examples, mnist.test.num_examples


data size  55000 5000 10000


In [9]:
print mnist.train.images[0]

[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         

In [10]:
print mnist.train.labels[0]

[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]


In [11]:
batch_size = 100
xs, ys = mnist.train.next_batch(batch_size)

print "X shape:", xs.shape

print "Y shape:", ys.shape

X shape: (100, 784)
Y shape: (100, 10)


In [15]:
# 定义常数集

# 输入层的节点数, 等于图片的像素
INPUT_NODE = 784
# 输出层的节点数, 等于类别的数目. 表示10个数字
OUTPUT_NODE = 10
# 配置神经网络的参数
# 隐藏层节点数, 这里使用只有一个隐藏层的网络结构作为样例, 这个隐藏层有500个节点
LAYER1_NODE = 500 
# 一个训练batch中的训练数据个数, 数字越小时, 训练过程越接近随机梯度下降, 数字越大越接近梯度下降
BATCH_SIZE = 100
# 基础的学习率
LEARNING_RATE_BASE = 0.8
# 学习率的衰减率
LEARNING_RATE_DECAY = 0.99
# 描述模型复杂度的正则化项在损失函数中的系数
REGULARIZATION_RATE = 0.0001
# 训练轮数
TRAINING_STEPS = 5000
# 滑动平均衰减率
MOVING_AVERAGE_DECAY = 0.99

# 一个辅助函数, 给定神经网络的输入和所有参数, 计算神经网络的前向传播结果. 
# 定义了一个使用RELU激活函数的三层全联接神经网络.通过加入隐藏层实现了多层网络结构
# 通过Relu激活函数实现了去线性化, 在这个函数中也支持传入用于计算参数平均值的类, 方便在测试时使用滑动平均模型
def inference(input_tensor, avg_class, weights1, biases1, weights2, biases2):
    # 当没有提供滑动平均类时, 直接使用参数当前的取值
    if avg_class == None:
        # 计算隐藏层的前向传播结果, 这里使用Relu激活函数
        layer1 = tf.nn.relu(tf.matmul(input_tensor, weights1) + biases1)
        
        # 计算输出层的前向传播结果, 因为在计算损失函数时会一并计算softmax函数, 所以这里不需要加入激活函数. 
        # 不加入softmax不会影响预测结果,因为预测时使用的是不同类别对应节点输出值的相对大小, 有没有softmax层对最后分类结果的计算没有影响.
        return tf.matmul(layer1, weights2) + biases2
    else:
        # 首先使用avg_class.average函数来计算得出变量的滑动平均值
        # 然后再计算相应的神经网络前向传播结果
        layer1 = tf.nn.relu(tf.matmul(input_tensor, avg_class.average(weights1)) + avg_class.average(biases1))
        return tf.matmul(layer1, avg_class.average(weights2)) + avg_class.average(biases2)

# 训练模型的过程
def train(mnist):
    x = tf.placeholder(tf.float32, [None, INPUT_NODE], name='x-input')
    y_ = tf.placeholder(tf.float32, [None, OUTPUT_NODE], name='y-input')
    # 生成隐藏层的参数
    weights1 = tf.Variable(tf.truncated_normal([INPUT_NODE, LAYER1_NODE], stddev=0.1))
    biases1 = tf.Variable(tf.constant(0.1, shape=[LAYER1_NODE]))
    # 生成输出层的参数
    weights2 = tf.Variable(tf.truncated_normal([LAYER1_NODE, OUTPUT_NODE], stddev=0.1))
    biases2 = tf.Variable(tf.constant(0.1, shape=[OUTPUT_NODE]))
    # 计算在当前参数下神经网络前向传播的结果.这里设置了计算滑动平均的类为None, 不会使用参数的滑动平均值
    y = inference(x, None, weights1, biases1, weights2, biases2)
    # 定义存储训练轮数的变量, 这个变量不需要计算滑动平均值, 所以这里指定这个变量为不可训练的变量(trainable=False)
    global_step = tf.Variable(0, trainable=False)
    # 给定滑动平均衰减率和训练轮数的变量, 初始化滑动平均类.
    variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)
    # 在所有代表神经网络参数的变量上使用滑动平均,辅助变量global_step就不需要了
    # tf.trainable_variables返回的就是图上集合Graphkeys.TRAINABLE_VARIABLES中的元素.这个集合的元素就是所有没有指定
    # trainable=False的参数
    variables_averages_op = variable_averages.apply(tf.trainable_variables())
    # 计算使用了滑动平均之后的前向传播结果,滑动平均不会改变变量本身的取值, 而是会维护一个影子变量来记其滑动平均值.
    average_y = inference(x, variable_averages, weights1, biases1, weights2, biases2)
    # 计算交叉熵作为刻画预测值和真实值之间差距的损失函数,这里使用了Tensorflow中sparse_softmax_cross_entropy_with_logits
    # 函数来计算交叉熵,当分类问题只有一个正确答案时, 可以使用这个函数来加速交叉熵的计算,MNIST问题的图片只有0-9中的一个数字
    # 可以使用这个函数来计算交叉熵损失, 第一个参数为神经网络不包括softmax层的前向传播结果, 第二个是训练数据的正确答案.
    # 因为标准答案是一个长度为10的一维数组, 而该函数需要提供的是一个正确答案的数字,所以需要使用tf.argmax函数来得到正确答案对应的分类编号
    # tf.argmax(input, dimension, name=None) dimension=0按行找,1按列找, 返回最大数值的下标
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=tf.argmax(y_, 1))
    # 计算在当前batch中所有样例的交叉熵平均值
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    
    # 计算L2正则化损失函数
    regularizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
    # 计算模型的正则化损失, 一般只计算神经网络边上权重的正则化损失, 而不使用偏置项
    regularization = regularizer(weights1) + regularizer(weights2)
    # 总损失等于交叉熵损失和正则化损失的和
    loss = cross_entropy_mean + regularization
    # 设置指数衰减的学习率
    learning_rate = tf.train.exponential_decay(
        LEARNING_RATE_BASE, # 基础学习率,随着迭代的进行,更新变量时使用的学习率在这个基础上递减
        global_step,        # 当前迭代的轮数
        mnist.train.num_examples / BATCH_SIZE, # 过完所有的训练数据需要的迭代次数
        LEARNING_RATE_DECAY # 学习率衰减速度
    )
    # 使用了tf.train.GradientDescentOptimizer优化算法来优化损失函数.包括交叉熵损失和L2正则化损失
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    # 在训练神经网络模型时, 每过一遍数据既需要通过反向传播来更新神经网络中的参数,又要更新每一个参数的滑动平均值.
    # 下面2行等价于 train_op = tf.group(train_step, variables_averages_op)
    with tf.control_dependencies([train_step, variables_averages_op]):
        train_op = tf.no_op(name='train')
    # 检验使用了滑动平均模型的神经网络前向传播结果是否正确. tf.argmax(average_y, 1)计算每一个样例的预测答案.其中average_y是一个
    # batch_size * 10 的二维数组, 每一行表示一个样例的前向传播结果. tf.argmax的第二个参数1表示选取最大值的操作仅在第一个纬度中进行.
    # 也就是说,只在每一行选取最大值对应的下标. 于是得到的结果是一个长度为batch的一维数组.这个一维数组中的值就表示了每一个样例对应的
    # 数字识别结果, tf.equal判断两个张量的每一维是否相等.
    correct_prediction = tf.equal(tf.argmax(average_y, 1), tf.argmax(y_, 1))
    # 这个运算首先将一个布尔型的数值转换为实数型,然后计算平均值.这个平均值就是模型在这一组数据上的正确率
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    # 初始化会话并开始训练过程
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        # 准备验证数据. 一般在神经网络的训练过程中会通过验证数据来大致判断停止的条件和评判训练的效果
        validate_feed = {x: mnist.validation.images, y_: mnist.validation.labels}
        # 准备测试数据, 在真实的应用中,这部分数据在训练时是不可见的, 这个数据只是作为模型优劣的最后评价标准
        test_feed = {x: mnist.test.images, y_: mnist.test.labels}
        # 迭代地训练神经网络
        for i in range(TRAINING_STEPS):
            if i % 1000 == 0:
                # 计算滑动平均模型在验证数据上的结果,因为MNIST数据集比较小,所以一次可以处理所有的验证数据.
                # 为了计算方便, 本样例程序没有将验证数据划分为更小的batch.当神经网络模型比较复杂或者验证数据比较大时,太大的batch
                # 会导致计算时间过长甚至发生内存溢出的错误
                validate_acc = sess.run(accuracy, feed_dict=validate_feed)
                print("after %d training step, validation accuracy using average model is %g" %(i, validate_acc))
            # 产生这一轮使用的一个batch的训练数据, 并运行训练过程
            xs, ys = mnist.train.next_batch(BATCH_SIZE)
            sess.run(train_op, feed_dict={x: xs, y_: ys})
        # 训练结束后,在测试数据上检测神经网络模型的最终正确率
        test_acc = sess.run(accuracy, feed_dict = test_feed)
        print("after %d training step, test accuracy using average model is %g" %(TRAINING_STEPS, test_acc))
        
def main(arggv=None):
    mnist = input_data.read_data_sets("./mnist/", one_hot=True)
    train(mnist)

if __name__ == '__main__':
    tf.app.run()

    


Extracting ./mnist/train-images-idx3-ubyte.gz
Extracting ./mnist/train-labels-idx1-ubyte.gz
Extracting ./mnist/t10k-images-idx3-ubyte.gz
Extracting ./mnist/t10k-labels-idx1-ubyte.gz
after 0 training step, validation accuracy using average model is 0.142
after 1000 training step, validation accuracy using average model is 0.9762
after 2000 training step, validation accuracy using average model is 0.9818
after 3000 training step, validation accuracy using average model is 0.9836
after 4000 training step, validation accuracy using average model is 0.9838
after 5000 training step, test accuracy using average model is 0.9823


SystemExit: 

/Users/xuning/anaconda3/envs/tf17/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
